## Fake News
Build a system to identify unreliable news articles (InClass Prediction Competition)

train.csv: A full training dataset with the following attributes:<br>

id: unique id for a news article<br>
title: the title of a news article<br>
author: author of the news article<br>
text: the text of the article; could be incomplete<br>
label: a label that marks the article as potentially unreliable<br>
1: unreliable<br>
0: reliable<br>
test.csv: A testing training dataset with all the same attributes at train.csv without the label.<br>
submit.csv: A sample submission that you can

Develop a machine learning program to identify when an article might be fake news. Run by the UTK Machine Learning Club.

Submission Format:
For every article in the test dataset, submission files should contain two columns: `id` and `label`. The `id` column should refer to a row in the `test.csv` file, and the `label` column should refer it's class of reliable (`0`), or potentially fake (`1`).

In [77]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib

### Exploaring the data

In [31]:
train_data= pd.read_csv('train.csv')
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [32]:
test_data = pd.read_csv('test.csv')
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [33]:
train_data.shape

(20800, 5)

In [34]:
test_data.shape

(5200, 4)

stropwords are those words which dosen't add much value to a paragraph or text can be the words such as (the, a , you , where, what ,when, etc...)

In [35]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [36]:
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [37]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
dtype: int64

### data Preprocessing

#### handling null values

In [38]:
train_data.fillna(' ', inplace=True)
test_data.fillna(' ', inplace=True)

test_data['content']=test_data['author']+' '+test_data['title']
train_data['content']=train_data['author']+' '+train_data['title']

In [39]:
train_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [40]:
test_data.head()

,id,title,author,text,content
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld Specter of Trump Loosens Tong...
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,Russian warships ready to strike terrorists ...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams #NoDAPL: Native American Leaders...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel Victor Tim Tebow Will Attempt Another C...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network Keiser Report: Meme Wa...


#### Stemming

In [41]:
port_stem = PorterStemmer()

In [42]:
def stemming(content):
    stemmed_content = re.sub('[^A-z]',' ',content) # this removes everything except Alphabets so it will removes numbers and sympoles and replace them by a space
    stemmed_content = stemmed_content.lower() # convert all words into lower case letters
    stemmed_content = stemmed_content.split() #Split the words into list
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # we are steamming this words which taking each word and reducing it to its root word and this case we are removing the stropwords
    stemmed_content = ' '.join(stemmed_content) # after finish joining all the words 
    return stemmed_content

In [43]:
train_data['content'] = train_data['content'].apply(stemming)
test_data['content'] = test_data['content'].apply(stemming)

In [44]:
train_data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [45]:
test_data['content']

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198                us marin deploy russian border norway
5199        teddi wayn awkward sex onscreen new york time
Name: content, Length: 5200, dtype: object

In [46]:
train_data['total'] = train_data['content'] + train_data['text']
train_data['total']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2        consortiumnew com truth might get fireWhy the ...
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799    david swanson keep f aliv  David Swanson is an...
Name: total, Length: 20800, dtype: object

In [47]:
test_data['total'] = test_data['content'] + test_data['text']
test_data['total']

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4       truth broadcast network keiser report meme war...
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198    us marin deploy russian border norway« Previou...
5199    teddi wayn awkward sex onscreen new york timeP...
Name: total, Length: 5200, dtype: object

#### converting the textual data to numerical data

In [48]:
transformer = TfidfTransformer(smooth_idf=False)
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
counts = count_vectorizer.fit_transform(train_data['total'].values)
tfidf = transformer.fit_transform(counts)

In [49]:
targets = train_data['label'].values
test_counts = count_vectorizer.transform(test_data['total'].values)
test_tfidf = transformer.fit_transform(test_counts)

C:\Users\eyada\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1453: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(n_samples / df) + 1


#### Assign Training and testing data

In [68]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, targets, random_state=50)

### Buliding and training the model

In [69]:
model = LogisticRegression(C=1e5)
model.fit(X_train, y_train)

C:\Users\eyada\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

### Accuracy of the model

In [71]:
model.score(X_train, y_train)

1.0

In [70]:
model.score(X_test, y_test)

0.9775

### Predict testing data

In [75]:
example_counts = count_vectorizer.transform(test_data['total'].values)
predictions = model.predict(example_counts)
pred=pd.DataFrame(predictions,columns=['label'])
pred['id']=test_data['id']
pred

,label,id
0,0,20800
1,0,20801
2,0,20802
3,0,20803
4,0,20804
...,...,...
5195,0,25995
5196,0,25996
5197,0,25997
5198,0,25998


### submission files

In [76]:
pred.to_csv('Sumbit.csv', index=False)

### save the model

In [78]:
joblib.dump(model, 'Fake News Detection')

['Fake News Detection']